In [1]:
# source: https://www.daniweb.com/programming/software-development/code/216839/number-to-word-converter-python
def int2word(n):
    """
    convert an integer number n into a string of english words
    """
    
    # Return any string that is not all digits
    if not all([char.isdigit() for char in n]):
        return n
    
    # break the number into groups of 3 digits using slicing
    # each group representing hundred, thousand, million, billion, ...
    n3 = []
    r1 = ""
    # create numeric string
    ns = str(n)
    for k in range(3, 33, 3):
        r = ns[-k:]
        q = len(ns) - k
        # break if end of ns has been reached
        if q < -2:
            break
        else:
            if  q >= 0:
                n3.append(int(r[:3]))
            elif q >= -1:
                n3.append(int(r[:2]))
            elif q >= -2:
                n3.append(int(r[:1]))
        r1 = r
    
    #print n3  # test
    
    # break each group of 3 digits into
    # ones, tens/twenties, hundreds
    # and form a string
    nw = ""
    for i, x in enumerate(n3):
        b1 = x % 10
        b2 = (x % 100)//10
        b3 = (x % 1000)//100
        #print b1, b2, b3  # test
        if x == 0:
            continue  # skip
        else:
            t = thousands[i]
        if b2 == 0:
            nw = ones[b1] + t + nw
        elif b2 == 1:
            nw = tens[b1] + t + nw
        elif b2 > 1:
            nw = twenties[b2] + ones[b1] + t + nw
        if b3 > 0:
            nw = ones[b3] + "hundred " + nw
    return nw.strip().split()

############# globals ################
ones = ["", "one ","two ","three ","four ", "five ",
    "six ","seven ","eight ","nine "]
tens = ["ten ","eleven ","twelve ","thirteen ", "fourteen ",
    "fifteen ","sixteen ","seventeen ","eighteen ","nineteen "]
twenties = ["","","twenty ","thirty ","forty ",
    "fifty ","sixty ","seventy ","eighty ","ninety "]
thousands = ["","thousand ","million ", "billion ", "trillion ",
    "quadrillion ", "quintillion ", "sextillion ", "septillion ","octillion ",
    "nonillion ", "decillion ", "undecillion ", "duodecillion ", "tredecillion ",
    "quattuordecillion ", "quindecillion", "sexdecillion ", "septendecillion ", 
    "octodecillion ", "novemdecillion ", "vigintillion "]

def digits_to_text(document):
    digits_to_text = []
    for token in document:
        temp = int2word(token)
        if type(temp) is list:
            digits_to_text.extend(temp)
        else:
            digits_to_text.append(temp)

    return digits_to_text

In [2]:
import torch
import pandas as pd
import os
import numpy as np
import json
import h5py
import heapq
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from pathlib import Path
import time
from timeit import default_timer as timer

import sys
import pprint
from collections import Counter,defaultdict
from itertools import chain

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

# TODO: find scientific reference that also claims Snowball is better
# alternatively: http://www.nltk.org/howto/stem.html claims this already.
from nltk.stem import SnowballStemmer, PorterStemmer

# check if stopword corpus is available on your system
try:
    _ = stopwords.words('english')
except:
    nltk.download('stopwords')
    
try:
    _ = WordNetLemmatizer().lemmatize('test')
except:
    nltk.download('wordnet')
    
# Embeddings don't work well for words that occur < 5 times
THRESHOLD = 1
UNK = "<unk>"

def hide_infrequent_words(document, threshold):
    counter = Counter(document)
    new_document = []
    
    for word in document:
        if counter[word] > threshold:
            new_document.append(word)
    
    return new_document

def filter_document(document):
    """Filter list of words based on some conventional methods, like removing stopwords and
    lemmatization"""

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    stop_words.update(punctuation)
    document = list(filter(lambda x: x not in stop_words, document))

    # [I, am, 34] -> [I, am, thirty, four]
    document = digits_to_text(document)

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    document = list(map(lemmatizer.lemmatize, document))

    return document

class DialogDataset(Dataset):
    
    def __init__(self, json_data, image_features, img2feat, transform=None):
        
            
        self.json_data = pd.read_json(json_data, orient='index')
        self.corpus = self.get_words()
        self.corpus = filter_document(self.corpus)
        self.corpus = hide_infrequent_words(self.corpus, THRESHOLD)
        self.vocab = list(set(self.corpus))
        
        self.vocab.append(UNK)
        
        self.w2i = {word : i for i, word in enumerate(self.vocab)}
        self.w2i = defaultdict(lambda: self.w2i[UNK], self.w2i)
        
        with open(img2feat, 'r') as f:
            img2feat = json.load(f)['IR_imgid2id']
        img_features = np.asarray(h5py.File(image_features, 'r')['img_features'])
        self.prepareData(img_features, img2feat)
        
    def prepareData (self, img_features, img2feat):
        all_img_features = []
        all_qa_vectors = []
        all_caption_vector = []
        all_dialog_vector = []
        for idx in range(len(self)):
            item = self.json_data.iloc[idx]
            image_features = [torch.FloatTensor(img_features[idx]) for idx in map(lambda x: img2feat[str(x)], item.img_list)]
            qa_vectors = [self.make_context_vector(line[0].split()) for line in item.dialog]
#             qa_embeddings = [sum(self.embeddings(Variable(qa_vector)).data) for qa_vector in qa_vectors]
            caption_vector = self.make_context_vector(item.caption.split())
#             caption_embedding = sum(self.embeddings(Variable(caption_vector)).data)
#             qa_embeddings.append(caption_embedding)
            dialog_vector = [word for line in item.dialog for word in line[0].split()]
            dialog_vector.extend(item.caption.split(' '))
            dialog_vector = self.make_context_vector(dialog_vector)
            
            all_img_features.append(image_features)
            all_qa_vectors.append(qa_vectors)
            all_caption_vector.append(caption_vector)
            all_dialog_vector.append(dialog_vector)
            
        self.json_data['img_features'] = all_img_features
        self.json_data['qa_vectors'] = all_qa_vectors
        self.json_data['caption_vector'] = all_caption_vector
        self.json_data['dialog_vector'] = all_dialog_vector
        
    # collect all the words from dialogs and 
    # captions and use them to create embedding map
    def get_words(self):
        words = []
        for idx in range(len(self)):
            item = self.json_data.iloc[idx]

            # Flatten dialog and add caption into 1d array
            dialog = [word for line in item.dialog for word in line[0].split()]
            dialog.extend(item.caption.split(' '))

            words.append(dialog)
            
        return list(chain.from_iterable(words))
    
    def make_context_vector(self, context):
        idxs = [self.w2i[w] for w in context]
        tensor = torch.LongTensor(idxs)
        return tensor

    def __len__(self):
        return len(self.json_data)

    def __getitem__(self, key):
        return self.json_data.iloc[key]
#         if isinstance(key, slice):
#             #Get the start, stop, and step from the slice
#             return [self[ii] for ii in range(*key.indices(len(self)))]
#         elif isinstance(key, int):
#             if key < 0 : #Handle negative indices
#                 key += len( self )
#             if key < 0 or key >= len(self) :
#                 raise IndexError("The index ({}) is out of range.".format(key))
            
#             item = self.json_data.iloc[key]

#             # Flatten dialog and add caption into 1d array
#             dialog = [word for line in item.dialog for word in line[0].split()]
#             dialog.extend(item.caption.split(' '))
#             dialog = filter_document(dialog)
#             dialog = self.make_context_vector(dialog)

#             img_ids = np.array(item.img_list)
#             img_features = [self.img_features[idx] for idx in map(lambda x: self.img2feat[str(x)], img_ids)]
#             img_features = np.array(img_features)
#             img_features = torch.FloatTensor(img_features)

#             target = item.target
#             target = torch.LongTensor(np.array([target]))

#             if torch.cuda.is_available():
#                 dialog, img_features, target = dialog.cuda(), img_features.cuda(), target.cuda()
                
#             return dialog, img_features, target

In [3]:
SAMPLE_EASY = ['Data', 'sample_easy.json']
TRAIN_EASY = ['Data', 'Easy', 'IR_train_easy.json']
EASY_1000 = ['Data', 'Easy', 'IR_train_easy_1000.json']
VAL_200 = ['Data', 'Easy', 'IR_val_easy_200.json']
VALID_EASY = ['Data', 'Easy', 'IR_val_easy.json']
IMG_FEATURES = ['Data', 'Features', 'IR_image_features.h5']
INDEX_MAP = ['Data', 'Features', 'IR_img_features2id.json']

IMG_SIZE = 2048
EMBEDDING_DIM = 2048

torch.manual_seed(1)
# dialog_data = DialogDataset(os.path.join(*SAMPLE_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))
dialog_data = DialogDataset(os.path.join(*TRAIN_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))
valid_data = DialogDataset(os.path.join(*VALID_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))

vocab_size = len(dialog_data.vocab)
print(len(dialog_data[0:3])) # can now slice this bitch up
dialog_data.json_data.head()

3


,caption,dialog,img_list,target,target_img_id,img_features,qa_vectors,caption_vector,dialog_vector
0,a person that is laying next to a dog,"[[is this a child or adult ? adult], [male or ...","[540370, 142686, 200718, 325806, 412575, 55676...",7,378466,"[[0.22707563638687134, 0.23830120265483856, 0....","[[11940, 11940, 11940, 8503, 11940, 4893, 1194...","[11940, 7729, 11940, 11940, 1696, 11697, 11940...","[11940, 11940, 11940, 8503, 11940, 4893, 11940..."
1,a black and white photo of a man on a horse by...,"[[what color is horse ? brown, but it's black ...","[502588, 414700, 509350, 303597, 575029, 43960...",4,575029,"[[1.5016311407089233, 0.18418647348880768, 0.7...","[[11940, 2277, 11940, 11591, 11940, 3008, 1194...","[11940, 292, 11940, 7431, 3674, 11940, 11940, ...","[11940, 2277, 11940, 11591, 11940, 3008, 11940..."
10,a cheese and pepperoni pizza in an oven,"[[is oven hot ? yes], [do u see red flames ? n...","[227779, 36324, 256451, 60043, 226587, 351705,...",1,36324,"[[0.06620161980390549, 0.3357774317264557, 1.9...","[[11940, 7731, 9910, 11940, 2169], [11940, 914...","[11940, 46, 11940, 8303, 2215, 1113, 11940, 7731]","[11940, 7731, 9910, 11940, 2169, 11940, 914, 1..."
100,an image of a kids playing with a frisbee,"[[how many kids are there ? 2], [what color is...","[234316, 197327, 474780, 247744, 456678, 14682...",9,100012,"[[1.5324183702468872, 0.2816334664821625, 0.51...","[[11940, 5368, 11940, 11940, 11940, 11940, 119...","[11940, 3264, 11940, 11940, 11940, 7521, 11940...","[11940, 5368, 11940, 11940, 11940, 11940, 1194..."
1000,2 people in a field throwing a frisbee at a a ...,"[[are the people female or male ? male], [abou...","[147838, 316269, 240195, 304038, 6146, 132638,...",6,219663,"[[0.5365859270095825, 0.39938515424728394, 1.2...","[[11940, 11940, 5362, 2800, 11940, 9062, 11940...","[11940, 5362, 1113, 11940, 5210, 5509, 11940, ...","[11940, 11940, 5362, 2800, 11940, 9062, 11940,..."


In [4]:
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, output_dim)
        self.activation_function1 = nn.ReLU()
        
#         #out: 1 x vocab_size
#         self.linear2 = nn.Linear(128, output_dim)
#         self.activation_function2 = nn.ReLU()
        

    def forward(self, inputs):
        # i believe .view() is useless here because the sum already produces a 1xEMB_DIM vector
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
#         out = self.activation_function1(out)
#         out = self.linear2(out)
#         out = self.activation_function2(out)
        return out

In [5]:
class MaxEnt(torch.nn.Module):
    
    def __init__(self, text_module, text_dim, img_size):
        super(MaxEnt, self).__init__()

        self.text_module = text_module
        
        self.inputDimension = text_dim + img_size
        self.hiddenDimension = 200
        self.outputDimension = 1
        self.numberOfLayers = 4
        self.linear1 = nn.Linear(self.inputDimension, self.hiddenDimension)
        self.linear2 = nn.Linear(self.hiddenDimension, self.hiddenDimension)
        self.linear3 = nn.Linear(self.hiddenDimension, self.hiddenDimension)
        self.lstm = nn.LSTM(
            input_size=self.hiddenDimension, 
            hidden_size=self.hiddenDimension,
            num_layers=self.numberOfLayers,
            batch_first=True
        )
        self.linear4 = nn.Linear(self.hiddenDimension, self.hiddenDimension)
        self.linear5 = nn.Linear(self.hiddenDimension, self.hiddenDimension)
        self.linear6 = nn.Linear(self.hiddenDimension, self.outputDimension)

        self.hidden = self.init_hidden(1)
#         self.softmax = nn.LogSoftmax()

        # for weightTensor in self.parameters():
        # 	weightTensor.data = torch.zeros(weightTensor.size())

    def init_hidden(self, batch_size):
        hidden1 = torch.zeros(self.numberOfLayers, batch_size, self.hiddenDimension)
        hidden2 = torch.zeros(self.numberOfLayers, batch_size, self.hiddenDimension)
        if torch.cuda.is_available():
            hidden1, hidden2 = hidden1.cuda(), hidden2.cuda()
        return (Variable(hidden1),Variable(hidden2))
        
    def prepare (self, item):
        list_of_string_vectors = []
        list_of_string_vectors.append(item.caption_vector)
        list_of_string_vectors.extend(item.qa_vectors)
        if torch.cuda.is_available():
            list_of_string_vectors = [vec.cuda() for vec in list_of_string_vectors]
        text_features = [self.text_module(Variable(vec, volatile=True)).data for vec in list_of_string_vectors]
        text_features = torch.squeeze(torch.stack(text_features))
#         print("text_features: ", text_features.size())
#         print(text_features[:9,:9])
        sequences = text_features.repeat(len(item.img_features), 1, 1)
#         print("sequences: ", sequences.size())
#         print(sequences[:3,:9,:9])
        img_add_on = torch.stack(item.img_features).repeat(text_features.size(0), 1, 1).permute(1,0,2)
        if torch.cuda.is_available():
            img_add_on = img_add_on.cuda()
#         print("img_add_on: ", img_add_on.size())
#         print(img_add_on[:3,:9,:9])
        sequences = torch.cat((sequences, img_add_on), 2)
#         print("final sequence: ", sequences.size())
#         print(sequences[:3,:9,:9])
        target = torch.LongTensor(np.array([item.target]))
        if torch.cuda.is_available():
            target = target.cuda()
        
        self.hidden = self.init_hidden(len(item.img_features))
        return Variable(sequences), Variable(target)
    
    def prepareBatch (self, batch):
        inputs = []
        targets = []
        for dialog, imgFeatures, target in batch:
            inputs.append(self.prepare(dialog, imgFeatures))
            targets.append(target)
        inputs = torch.cat(inputs)
        targets = torch.cat(targets)
        return Variable(inputs, requires_grad=True), Variable(targets)
        
    def forward(self, inp, batch_size):
#         print(inp.size())
        scores = self.linear1(inp)
        scores = F.relu(scores)
        scores = self.linear2(scores)
        scores = F.relu(scores)
        scores = self.linear3(scores)
        scores = F.relu(scores)
        scores, self.hidden = self.lstm(scores, self.hidden)
        scores = F.relu(scores)
        scores = self.linear4(scores)
        scores = F.relu(scores)
        scores = self.linear5(scores)
        scores = F.relu(scores)
        scores = self.linear6(scores)
        scores = F.relu(scores)
#         lstmOut = F.sigmoid(lstmOut)
#         print(lstmOut.size())
#         scores = self.linear(lstmOut).view(batch_size, -1)
#         scores = F.sigmoid(scores)
#         print(scores.size())
#         print(scores)
        scores = F.log_softmax(scores.squeeze(), dim=0)
#         print(scores.size())
        return scores

In [6]:
TEXT_DIM = 2048

cbow_model = CBOW(vocab_size, EMBEDDING_DIM, TEXT_DIM)
model = MaxEnt(cbow_model, TEXT_DIM, IMG_SIZE)

if torch.cuda.is_available():
    print("ya ya")
    cbow_model = cbow_model.cuda()
    model = model.cuda()
    print("cuda ready bitches")
else:
    print("no no")
    
training_errors = []
validation_errors = []
epochs_trained = 0

no no


In [7]:
def validate(model, data, loss_func):
    total_loss = 0
    item_count = 0
    for i, item in data.iterrows():
        inputs, target = model.prepare(item)
        pred = model(inputs, len(item.img_features))
        print(pred)
        pred = pred[-1,:].unsqueeze(0)
        loss = loss_func(pred, target)
        total_loss += loss.data[0]
        break
    return total_loss / len(data)

def predict(model, data):
    correct_top1 = 0
    correct_top5 = 0
    
    for i, item in data.iterrows():
        
        inputs, target = model.prepare(item)
        
        # For top 1:
        pred = model(inputs, len(item.img_features))
        pred = pred[:,-1].unsqueeze(0)
        img, idx = torch.max(pred, 1)

        if idx.data[0] == target.data[0]:
            correct_top1 += 1
        
        # For top 5:
        pred = pred.data.cpu().numpy().flatten()
        top_5 = heapq.nlargest(5, range(len(pred)), pred.__getitem__)
        if target.data[0] in top_5:
            correct_top5 += 1
    
    return correct_top1 / len(data), correct_top5 / len(data)

validate(model, valid_data[:100], nn.NLLLoss())

Variable containing:

Columns 0 to 9 
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026
-2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026 -2.3026

Columns 10 to 10 
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
-2.3026
[torch.FloatTensor of size 10x11]



0.023025963306427002

In [8]:
def log_to_console(i, n_epochs, batch_size, batch_per_epoch, error, start_time, processing_speed):
    percentOfEpc = (i / batch_per_epoch) * 100
    print("{:.0f}s:\t epoch: {}\t batch:{} ({:.1f}%) \t training error: {:.6f}\t speed: {:.1f} dialogs/s"
          .format(timer() - start_time, 
                  n_epochs, 
                  i, 
                  percentOfEpc, 
                  error, 
                  processing_speed))
    
def init_stats_log(label, training_portion, validation_portion, embeddings_dim, epochs, batch_count):
    timestr = time.strftime("%m-%d-%H-%M")
    filename = "{}-t_size_{}-v_size_{}-emb_{}-eps_{}-dt_{}-batch_{}.txt".format(label,
                                                                       training_portion,
                                                                       validation_portion,
                                                                       EMBEDDING_DIM,
                                                                       epochs,
                                                                       timestr,
                                                                       batch_count)

    target_path = ['Training_recordings', filename]
    stats_log = open(os.path.join(*target_path), 'w')
    stats_log.write("EPOCH|AVG_LOSS|TOT_LOSS|VAL_ERROR|CORRECT_TOP1|CORRECT_TOP5\n")
    print("Logging enabled in:", filename)
    
    return stats_log, filename
    

In [9]:
batch_size = 1 # batching is not possible with LSTM model since we use batching for different images
numEpochs = 100
learningRate = 1e-2
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learningRate)

start_time = timer()
lastPrintTime = start_time
continueFromEpc = 0
continueFromI = 0
sampleCount = len(dialog_data)
batchCountPerEpc = int(sampleCount/batch_size)-1
remainderCount = sampleCount - batchCountPerEpc * batch_size
print("we have: {} dialogs, batch size of {} with {} as remainder to offset batches each epoch".format(sampleCount, batch_size, remainderCount))
offset = 0

logging = True

training_portion = len(dialog_data)
validation_portion = len(valid_data)

if logging == True:
    stats_log, filename = init_stats_log("test_top1_top2", 
                               training_portion,
                               validation_portion,
                               EMBEDDING_DIM,
                               numEpochs,
                               batch_size)

else:
    print("Logging disabled!")
    filename = ""

for t in range(numEpochs):
    lastPrintTime = timer()
    epoch_start_time = timer()
    
    total_loss = 0
    updates = 0
    
    if t == 0 and continueFromI > 0:
        # continue where I crashed
        print("continuing")
#         model.load_state_dict(torch.load('maxent_{}epc_{}iter.pt'.format(continueFromEpc, continueFromI+1)))
    
    for i in range(continueFromI, batchCountPerEpc):
        
               
        batchBegin = offset + i * batch_size
        batchEnd = batchBegin + batch_size
        
        item = dialog_data[i]
        
        inputs, target = model.prepare(item)
        pred = model(inputs, len(item.img_features))
        loss = criterion(pred[-1,:].unsqueeze(0), target)

        training_errors.append(loss.data[0])
        total_loss += loss.data[0]
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        if timer()  - lastPrintTime > 3:
            processing_speed = (i*batch_size) / (timer() - epoch_start_time)
            log_to_console(i, epochs_trained, batch_size, batchCountPerEpc, total_loss / i, start_time, processing_speed)
            lastPrintTime = timer()
    print("{:.1f}s:\t Finished epoch. Calculating test error..".format(timer() - start_time))
    
    avg_loss = total_loss / batchCountPerEpc
    top_1_score, top_5_score = predict(model, valid_data)
    validation_error = validate(model, valid_data, criterion)
    
    if logging == True:
        stats_log.write("{}|{}|{}|{}|{}|{}\n".format(epochs_trained, avg_loss, total_loss, validation_error, top_1_score, top_5_score))
            
    epochs_trained += 1
    offset = (offset + 1) % remainderCount
    print("{:.1f}s:\t test error: {:.6f}".format(timer() - start_time, validation_error))
    continueFromI = 0
    if epochs_trained % 10 == 0:
        fileName = "maxent_{}batch_{}epc.pt".format(batch_size, epochs_trained)
        torch.save(model.state_dict(), fileName)
        print("saved\t", fileName)

for i in range(batch_size):
    dialog, image, target = dialog_data[i]
    inputs = model.prepare(dialog, image)
    pred = model(Variable(inputs))
    print(pred)

if logging == True:
    stats_log.close()

we have: 40000 dialogs, batch size of 1 with 1 as remainder to offset batches each epoch
Logging enabled in: test_top1_top2-t_size_40000-v_size_5000-emb_2048-eps_100-dt_12-17-16-19-batch_1.txt
3s:	 epoch: 0	 batch:14 (0.0%) 	 training error: 2.467033	 speed: 4.4 dialogs/s
6s:	 epoch: 0	 batch:30 (0.1%) 	 training error: 2.379328	 speed: 4.9 dialogs/s
9s:	 epoch: 0	 batch:45 (0.1%) 	 training error: 2.353747	 speed: 4.9 dialogs/s
12s:	 epoch: 0	 batch:61 (0.2%) 	 training error: 2.340327	 speed: 4.9 dialogs/s
15s:	 epoch: 0	 batch:77 (0.2%) 	 training error: 2.332485	 speed: 5.0 dialogs/s
19s:	 epoch: 0	 batch:92 (0.2%) 	 training error: 2.327610	 speed: 4.9 dialogs/s
22s:	 epoch: 0	 batch:107 (0.3%) 	 training error: 2.324102	 speed: 4.9 dialogs/s
25s:	 epoch: 0	 batch:122 (0.3%) 	 training error: 2.321456	 speed: 4.9 dialogs/s
28s:	 epoch: 0	 batch:138 (0.3%) 	 training error: 2.319268	 speed: 5.0 dialogs/s
31s:	 epoch: 0	 batch:154 (0.4%) 	 training error: 2.317535	 speed: 5.0 dialog

306s:	 epoch: 0	 batch:1155 (2.9%) 	 training error: 2.304578	 speed: 3.8 dialogs/s
309s:	 epoch: 0	 batch:1165 (2.9%) 	 training error: 2.304561	 speed: 3.8 dialogs/s
312s:	 epoch: 0	 batch:1175 (2.9%) 	 training error: 2.304545	 speed: 3.8 dialogs/s
316s:	 epoch: 0	 batch:1185 (3.0%) 	 training error: 2.304528	 speed: 3.8 dialogs/s
319s:	 epoch: 0	 batch:1195 (3.0%) 	 training error: 2.304512	 speed: 3.7 dialogs/s
322s:	 epoch: 0	 batch:1205 (3.0%) 	 training error: 2.304496	 speed: 3.7 dialogs/s
325s:	 epoch: 0	 batch:1215 (3.0%) 	 training error: 2.304480	 speed: 3.7 dialogs/s
328s:	 epoch: 0	 batch:1225 (3.1%) 	 training error: 2.304465	 speed: 3.7 dialogs/s
331s:	 epoch: 0	 batch:1235 (3.1%) 	 training error: 2.304449	 speed: 3.7 dialogs/s
334s:	 epoch: 0	 batch:1245 (3.1%) 	 training error: 2.304434	 speed: 3.7 dialogs/s
337s:	 epoch: 0	 batch:1255 (3.1%) 	 training error: 2.304420	 speed: 3.7 dialogs/s
341s:	 epoch: 0	 batch:1265 (3.2%) 	 training error: 2.304405	 speed: 3.7 di

613s:	 epoch: 0	 batch:2134 (5.3%) 	 training error: 2.303664	 speed: 3.5 dialogs/s
616s:	 epoch: 0	 batch:2144 (5.4%) 	 training error: 2.303659	 speed: 3.5 dialogs/s
620s:	 epoch: 0	 batch:2154 (5.4%) 	 training error: 2.303654	 speed: 3.5 dialogs/s
623s:	 epoch: 0	 batch:2164 (5.4%) 	 training error: 2.303649	 speed: 3.5 dialogs/s
626s:	 epoch: 0	 batch:2174 (5.4%) 	 training error: 2.303644	 speed: 3.5 dialogs/s
629s:	 epoch: 0	 batch:2184 (5.5%) 	 training error: 2.303639	 speed: 3.5 dialogs/s
632s:	 epoch: 0	 batch:2194 (5.5%) 	 training error: 2.303634	 speed: 3.5 dialogs/s
635s:	 epoch: 0	 batch:2204 (5.5%) 	 training error: 2.303630	 speed: 3.5 dialogs/s
638s:	 epoch: 0	 batch:2214 (5.5%) 	 training error: 2.303625	 speed: 3.5 dialogs/s
641s:	 epoch: 0	 batch:2224 (5.6%) 	 training error: 2.303620	 speed: 3.5 dialogs/s
644s:	 epoch: 0	 batch:2234 (5.6%) 	 training error: 2.303616	 speed: 3.5 dialogs/s
648s:	 epoch: 0	 batch:2244 (5.6%) 	 training error: 2.303611	 speed: 3.5 di

922s:	 epoch: 0	 batch:3116 (7.8%) 	 training error: 2.303324	 speed: 3.4 dialogs/s
925s:	 epoch: 0	 batch:3126 (7.8%) 	 training error: 2.303322	 speed: 3.4 dialogs/s
928s:	 epoch: 0	 batch:3136 (7.8%) 	 training error: 2.303319	 speed: 3.4 dialogs/s
931s:	 epoch: 0	 batch:3146 (7.9%) 	 training error: 2.303317	 speed: 3.4 dialogs/s
934s:	 epoch: 0	 batch:3155 (7.9%) 	 training error: 2.303315	 speed: 3.4 dialogs/s
937s:	 epoch: 0	 batch:3165 (7.9%) 	 training error: 2.303313	 speed: 3.4 dialogs/s
940s:	 epoch: 0	 batch:3175 (7.9%) 	 training error: 2.303310	 speed: 3.4 dialogs/s
943s:	 epoch: 0	 batch:3185 (8.0%) 	 training error: 2.303308	 speed: 3.4 dialogs/s
946s:	 epoch: 0	 batch:3195 (8.0%) 	 training error: 2.303306	 speed: 3.4 dialogs/s
950s:	 epoch: 0	 batch:3205 (8.0%) 	 training error: 2.303303	 speed: 3.4 dialogs/s
953s:	 epoch: 0	 batch:3215 (8.0%) 	 training error: 2.303301	 speed: 3.4 dialogs/s
956s:	 epoch: 0	 batch:3225 (8.1%) 	 training error: 2.303299	 speed: 3.4 di

1225s:	 epoch: 0	 batch:4085 (10.2%) 	 training error: 2.303149	 speed: 3.3 dialogs/s
1229s:	 epoch: 0	 batch:4095 (10.2%) 	 training error: 2.303147	 speed: 3.3 dialogs/s
1232s:	 epoch: 0	 batch:4106 (10.3%) 	 training error: 2.303146	 speed: 3.3 dialogs/s
1235s:	 epoch: 0	 batch:4116 (10.3%) 	 training error: 2.303144	 speed: 3.3 dialogs/s
1238s:	 epoch: 0	 batch:4127 (10.3%) 	 training error: 2.303143	 speed: 3.3 dialogs/s
1241s:	 epoch: 0	 batch:4138 (10.3%) 	 training error: 2.303141	 speed: 3.3 dialogs/s
1245s:	 epoch: 0	 batch:4148 (10.4%) 	 training error: 2.303140	 speed: 3.3 dialogs/s
1248s:	 epoch: 0	 batch:4158 (10.4%) 	 training error: 2.303139	 speed: 3.3 dialogs/s
1251s:	 epoch: 0	 batch:4167 (10.4%) 	 training error: 2.303138	 speed: 3.3 dialogs/s
1254s:	 epoch: 0	 batch:4177 (10.4%) 	 training error: 2.303136	 speed: 3.3 dialogs/s
1257s:	 epoch: 0	 batch:4187 (10.5%) 	 training error: 2.303135	 speed: 3.3 dialogs/s
1260s:	 epoch: 0	 batch:4197 (10.5%) 	 training error:

...

2428s:	 epoch: 0	 batch:7940 (19.9%) 	 training error: 2.302875	 speed: 3.3 dialogs/s
2431s:	 epoch: 0	 batch:7950 (19.9%) 	 training error: 2.302875	 speed: 3.3 dialogs/s
2434s:	 epoch: 0	 batch:7960 (19.9%) 	 training error: 2.302874	 speed: 3.3 dialogs/s
2437s:	 epoch: 0	 batch:7970 (19.9%) 	 training error: 2.302874	 speed: 3.3 dialogs/s
2440s:	 epoch: 0	 batch:7981 (20.0%) 	 training error: 2.302874	 speed: 3.3 dialogs/s
2443s:	 epoch: 0	 batch:7991 (20.0%) 	 training error: 2.302873	 speed: 3.3 dialogs/s
2446s:	 epoch: 0	 batch:8001 (20.0%) 	 training error: 2.302873	 speed: 3.3 dialogs/s
2450s:	 epoch: 0	 batch:8011 (20.0%) 	 training error: 2.302873	 speed: 3.3 dialogs/s
2453s:	 epoch: 0	 batch:8021 (20.1%) 	 training error: 2.302872	 speed: 3.3 dialogs/s
2456s:	 epoch: 0	 batch:8031 (20.1%) 	 training error: 2.302872	 speed: 3.3 dialogs/s
2459s:	 epoch: 0	 batch:8041 (20.1%) 	 training error: 2.302871	 speed: 3.3 dialogs/s
2462s:	 epoch: 0	 batch:8051 (20.1%) 	 training error:

2728s:	 epoch: 0	 batch:8904 (22.3%) 	 training error: 2.302844	 speed: 3.3 dialogs/s
2731s:	 epoch: 0	 batch:8914 (22.3%) 	 training error: 2.302843	 speed: 3.3 dialogs/s
2734s:	 epoch: 0	 batch:8924 (22.3%) 	 training error: 2.302843	 speed: 3.3 dialogs/s
2737s:	 epoch: 0	 batch:8934 (22.3%) 	 training error: 2.302843	 speed: 3.3 dialogs/s
2741s:	 epoch: 0	 batch:8944 (22.4%) 	 training error: 2.302843	 speed: 3.3 dialogs/s
2744s:	 epoch: 0	 batch:8954 (22.4%) 	 training error: 2.302842	 speed: 3.3 dialogs/s
2747s:	 epoch: 0	 batch:8964 (22.4%) 	 training error: 2.302842	 speed: 3.3 dialogs/s
2750s:	 epoch: 0	 batch:8974 (22.4%) 	 training error: 2.302842	 speed: 3.3 dialogs/s
2753s:	 epoch: 0	 batch:8984 (22.5%) 	 training error: 2.302841	 speed: 3.3 dialogs/s
2756s:	 epoch: 0	 batch:8994 (22.5%) 	 training error: 2.302841	 speed: 3.3 dialogs/s
2759s:	 epoch: 0	 batch:9004 (22.5%) 	 training error: 2.302841	 speed: 3.3 dialogs/s
2762s:	 epoch: 0	 batch:9014 (22.5%) 	 training error:

3027s:	 epoch: 0	 batch:9863 (24.7%) 	 training error: 2.302819	 speed: 3.3 dialogs/s
3030s:	 epoch: 0	 batch:9873 (24.7%) 	 training error: 2.302818	 speed: 3.3 dialogs/s
3033s:	 epoch: 0	 batch:9883 (24.7%) 	 training error: 2.302818	 speed: 3.3 dialogs/s
3036s:	 epoch: 0	 batch:9893 (24.7%) 	 training error: 2.302818	 speed: 3.3 dialogs/s
3039s:	 epoch: 0	 batch:9903 (24.8%) 	 training error: 2.302818	 speed: 3.3 dialogs/s
3043s:	 epoch: 0	 batch:9913 (24.8%) 	 training error: 2.302817	 speed: 3.3 dialogs/s
3046s:	 epoch: 0	 batch:9923 (24.8%) 	 training error: 2.302817	 speed: 3.3 dialogs/s
3049s:	 epoch: 0	 batch:9933 (24.8%) 	 training error: 2.302817	 speed: 3.3 dialogs/s
3052s:	 epoch: 0	 batch:9943 (24.9%) 	 training error: 2.302817	 speed: 3.3 dialogs/s
3055s:	 epoch: 0	 batch:9953 (24.9%) 	 training error: 2.302816	 speed: 3.3 dialogs/s
3058s:	 epoch: 0	 batch:9963 (24.9%) 	 training error: 2.302816	 speed: 3.3 dialogs/s
3062s:	 epoch: 0	 batch:9974 (24.9%) 	 training error:

3325s:	 epoch: 0	 batch:10816 (27.0%) 	 training error: 2.302798	 speed: 3.3 dialogs/s
3328s:	 epoch: 0	 batch:10826 (27.1%) 	 training error: 2.302798	 speed: 3.3 dialogs/s
3331s:	 epoch: 0	 batch:10836 (27.1%) 	 training error: 2.302798	 speed: 3.3 dialogs/s
3334s:	 epoch: 0	 batch:10846 (27.1%) 	 training error: 2.302797	 speed: 3.3 dialogs/s
3337s:	 epoch: 0	 batch:10856 (27.1%) 	 training error: 2.302797	 speed: 3.3 dialogs/s
3340s:	 epoch: 0	 batch:10866 (27.2%) 	 training error: 2.302797	 speed: 3.3 dialogs/s
3343s:	 epoch: 0	 batch:10876 (27.2%) 	 training error: 2.302797	 speed: 3.3 dialogs/s
3347s:	 epoch: 0	 batch:10886 (27.2%) 	 training error: 2.302797	 speed: 3.3 dialogs/s
3350s:	 epoch: 0	 batch:10896 (27.2%) 	 training error: 2.302796	 speed: 3.3 dialogs/s
3353s:	 epoch: 0	 batch:10906 (27.3%) 	 training error: 2.302796	 speed: 3.3 dialogs/s
3356s:	 epoch: 0	 batch:10917 (27.3%) 	 training error: 2.302796	 speed: 3.3 dialogs/s
3359s:	 epoch: 0	 batch:10927 (27.3%) 	 tra

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def draw_graph(filename):
    
    
    # Read file and data
    with open("Training_recordings/" + filename, 'r') as f:
        data = [x.strip() for x in f.readlines()] 
    
    data = np.array([line.split("|") for line in data[1:]]).T
    
    epochs, avg_loss, total_loss, val_error, correct_top_1, correct_top_5 = data
    
    epochs = np.array(epochs, dtype=np.int8)
    
    plt.subplot(4, 1, 1)
    plt.plot(epochs, np.array(avg_loss, dtype=np.float32), '.-')
    plt.title('average loss, validation error and correct predictions')
    plt.ylabel('Average\nLoss')
    plt.xlabel('Epochs')

    plt.subplot(4, 1, 2)
    plt.plot(epochs, np.array(val_error, dtype=np.float32), '-')
    plt.ylabel('Validation\nLoss')
    plt.xlabel('Epochs')

    plt.subplot(4, 1, 3)
    plt.plot(epochs, np.array(correct_top_1, dtype=np.int8), '-')
    plt.ylabel('Correct\ntop 1')
    plt.xlabel('Epochs')

    plt.subplot(4, 1, 4)
    plt.plot(epochs, np.array(correct_top_5, dtype=np.int8), '-')
    plt.ylabel('Correct\ntop 5')
    plt.xlabel('Epochs')

    
    
    plt.show()

draw_graph(filename)